#### Myrela Bauman  // May 2023
## This code loads all password protected Daily SPOT reports saved in the folder, combining all files into a data frame for analysis

In [ ]:
# installing library
#! pip install msoffcrypto-tool

# loading packages
import os
import pandas as pd
import msoffcrypto
import io
from msoffcrypto.exceptions import FileFormatError
from zipfile import BadZipFile
from msoffcrypto import OfficeFile

In [ ]:
# get user id - this links to the {uid} filepath below so that any user can run this code
uid = !id -u
uid = uid[0]
uid

In [ ]:
# create vars for folder path and password
folder_path = f'/run/user/{uid}/gvfs/smb-share:server=nasprgshare220,share=share/!!file path here!!'
password = "!! password here !!"

In [ ]:
# list all xlsx files in the folder
file_list = [f for f in os.listdir(f'{folder_path}') if f.endswith(".xlsx")]

file_list

In [ ]:
# initializing an empty DataFrame
combined_data = pd.DataFrame()

In [ ]:
# looping through each file in the folder
for file in file_list:
    file_path = os.path.join(folder_path, file)
    
    with open(file_path, "rb") as f:
        decrypted_file = None
        
        try:
            # try to decrypt the password-protected excel file
            decryptor = OfficeFile(f)
            decryptor.load_key(password=password)
            
            decrypted_file = io.BytesIO()
            decryptor.decrypt(decrypted_file)
            decrypted_file.seek(0)
        except FileFormatError:
            # if it's not an encrypted excel file, pass the unencrypted excel file to pandas
            f.seek(0)
            decrypted_file = f
        
        try:
            # read the decrypted data (or original data, if not encrypted) using pandas
            data = pd.read_excel(decrypted_file, sheet_name="!! sheet name here !!") #edit sheet name
            combined_data = pd.concat([combined_data, data], ignore_index=True)
        except Exception as e:
            print(f"Error processing file {file}: {e}")


In [ ]:
# NOTE: if you get the error message below when reading the files, it's because these files are restricted and can't be read
# Error processing file Potential Suspected Overdoses 2.10.2023.xlsx: Can't find workbook in OLE2 compound document
# Error processing file Potential Suspected Overdoses 1-11-2023.xlsx: Can't find workbook in OLE2 compound document

# printing dataframe to see if code worked
print(combined_data.head())

In [ ]:
# printing all variable names and data types
print(combined_data.info())

In [ ]:
# restricting to variables of interest
selected_columns = [
    "DOD", "REPORT DATE", "OCME NUMBER", "AGE", "RACE", "GENDER", 
    "DEATH ZIP", "HOME ZIP", "FOUND ZIP", "DEATH ADDRESS", "HOME ADDRESS",
    "FOUND ADDRESS", "NOTES"
]

filtered_data = combined_data[selected_columns]

print(filtered_data.head())

# if needed, can save the restricted dataset as an xlsx file:
# filtered_data.to_excel('combined_OCMESPOT_5.12.23.xlsx', index=False, engine="openpyxl")